## Introdução

O M-LOOP é uma ferramente de aplicação de métodos de aprendizado de máquina para a otimização de parâmetros de experimentos científicos ou outros sistemas controlados por um computador. O nome em si é uma sigla para Machine Learning Online Optimization Package, onde "Online Optimization" indica que a otimização é executada enquanto o experimento está sendo rodado, podendo então otimizar o equipamento "ao vivo". 

A ferramenta pode ser aplicada como um simples executável, através do terminal ou como uma biblioteca do Python, sendo esta última abordagem aquela que adotaremos. O pacote foi utilizado pela primeira vez em 

> Fast Machine-Learning Online Optimization of Ultra-Cold-Atom Experiments. Scientific Reports 6, 25890 (2016). DOI: [Link 10.1038/srep25890](https://www.nature.com/articles/srep25890)

E é mantido por Michael Hush, que fez parte da publicação original. O código original incluía como métodos de seleção de parâmetros um seletor aleatório e métodos de Noldor-Mead, Evolução Diferencial e Processo Gaussiano. Em 2018 o condigo foi expandido para incluir um learner de rede neural, na seguinte publicação 

> Tranter, A.D., Slatyer, H.J., Hush, M.R. et al. Multiparameter optimisation of a magneto-optical trap using deep learning. Nat Commun 9, 4360 (2018). DOI: [Link 10.1038/s41467-018-06847-1](https://doi.org/10.1038/s41467-018-06847-1)

Que é a versão que usaremos.

## M-LOOP como biblioteca do Python

A biblioteca do M-LOOP está dividida em 7 módulos, dos quais apenas três são chamados pelo usuário na escrita do script que executa o experimento e a otimização. Ainda assim, discutiremos a finalidade e alguns detalhes de cada um dos módulos antes de passarmos à aplicação da ferramenta. A informação completa sobre cada componente do pacote pode ser encontrada na documentação da versão expandida.

#### `mloop.controllers`

Os controladores fazem como diz o nome: são responsáveis por controlar a otimização, a passagem dos parâmetros ao experimento e chamar a execução do experimento em si.

*classe* `mloop.controllers.controller`  
Classe principal dos controladores, sem fazer referência a um learner específico. Cada learner tem sua própria classe que é subclasse desta (ou seja, herda seus parâmetros). Tem como argumentos
- `max_num_runs`, `max_num_runs_without_better_params`, `target_cost`: condições de parada (número máximo de execuções, número máximo de execuções sem queda do custo, custo alvo)
- `controller_archive_filename`, `controller_archive_file_type`: formato e nome dos arquivos da execução (registram dados de cada passo da otimização), variáveis extras a serem salvas nos arquivos
- `start_datetime`: horário de início da execução. Recebe também uma **interface**, o próximo módulo a ser detalhado.

A Evolução Diferencial herda diretamente dessa classe.

*classe* `mloop.controllers.DifferentialEvolutionController`  
- Não tem argumentos extras.

Para os métodos de otimização baseados em aprendizado de máquina, temos uma subclasse geral da classe de controladores geral.

*classe* `mloop.controllers.MachineLearnerController`  
Além dos argumentos que herda,
- `training_type`: os métodos de aprendizado de máquina precisam de um conjunto de dados iniciais (parâmetros e custo resultante) com os quais treinar o algoritmo para que ele possa gerar um primeiro conjunto de parâmetros com o qual alimentar o experimento. Estes dados iniciais são gerados por um método não baseado em aprendizado de máquina, um algoritmo de treino, que deve ser selecionado aqui: `random`, `nelder_mead`, `differential_evolution`. Este método também será chamado caso o machine learner esteja lento demais e o argumento `no_delay` seja `True`. **default**: `differential_evolution`
- `num_training_runs`: numero de execuções do algoritmo de treino antes da primeira chamada do machine learner. **default**: 10 ou duas vezes o número de parâmetros, o que for maior.
- `no_delay`: O machine learner pode às vezes demorar mais a treinar seu modelo e emitir os novos parâmetros, o que pode causar um atraso entre o retorno do último custo e a emissão dos novos parâmetros. Se este argumento for `True`, o atraso é evitado chamando o algoritmo de treino quanto o treino do machine learner se atrasa. Se `False`, o controlador aguarda que o machine learner complete a atualização de seu modelo interno.

Temos dois machine learners, subclasses de `mloop.controllers.MachineLearnerController`

*classe* `mloop.controllers.GaussianProcessController`
- Não tem argumentos extras.

*classe* `mloop.controllers.NeuralNetController`
- Não tem argumentos extras.

Além dos outros algoritmos de seleção/otimização

*classe* `mloop.controllers.DifferentialEvolutionController`
- Não tem argumentos extras.

*classe* `mloop.controllers.NelderMeadController`
- Não tem argumentos extras.

*classe* `mloop.controllers.RandomController`
- Não tem argumets extras.

#### `mloop.interfaces`

A interface é o meio utilizado pelo controlador para conectar o experimento à otimização. É onde pode ser feito qualquer tipo de pré-processamento nas medidas resultantes do experimento para a criação de um custo final a ser transmitido ao otimizador. Faz uso constante de `queues`

`queue`: uma estrutura de dados linear, que faz armazenamento do tipo "First In First Out" (FIFO), como em uma fila. 

Dentro das interfaces, objetos `queue` são usados para armazenar parâmetros que devem ser passados ao experimento, e custos que devem ser retornados ao otimizados, ambos casos onde é importante manter uma ordem fixa quanto ao significado dos números, daí a utilidade de `queues`.

*classe* `mloop.interfaces.Interface`  
Classe de interfaces que preenche `costs_in_queue` e lê `params_out_queue`. Subclasse de `threading.Thread`. Contém o único método da interface que precisa ser chamado explicitamente pelo usuário  
*método* `get_next_cost_dict`(*params_dict*)  
Parâmetros:
- `params_dict`: dicionário contendo os parâmetros retornados pelo otimizador sob a etiqueta 'params'  
Retorna:
- `cost_dict`: Contém as propriedades do experimento retornadas após a execução. Por padrão, pode conter *'cost'*, o custo em si, *'uncer'*, sua incerteza, e *'bad'*, um flag binário que indica se um resultado foi completamente inadequado. Pode conter também qualquer outra entrada adicionada pelo usuário, que será armazenada pelo controlador e salva nos arquivos de registro das execuções  

*classe* `mloop.interfaces.FileInterface`  
Subclasse de `mloop.interfaces.Interface` para a implementação do M-LOOP por arquivos. Nesta implementação, o M-LOOP cria um arquivo de formato escolhido contendo os parâmetros otimizados, que devem ser lidos pelo laboratório, e então aguarda a criação de um arquivo de custo, que deve ter os mesmos itens que o dicionário `cost_dict`. Ao detectar o arquivo de custo, o M-LOOP usa os novos dados para atualizar seu modelo antes de criar um novo arquivo de parâmetros. Tem como argumentos:
- `interface_out_filename`: nome do arquivo com os parâmetros otimizados.
- `interface_in_filename`: nome do arquivo com o custo
- `interface_file_type`: 'txt' para um arquivo de texto ou 'mat' para um arquivo do Matlab.  

*classe* `mloop.interfaces.ShellInterface`  
Subclasse de `mloop.interfaces.Interface` para a implementação do M-LOOP pelo terminal. Esta implementação pode ser utilizada quando o experimento pode ser executado por meio de um comando no terminal, que é executado pelo próprio M-LOOP. Tem como argumentos:
- `command`: o comando que o M-LOOP deve utilizar para executar o experimento. 
- `params_arg_type`: formato no qual o M-LOOP deve passar os parâmetros otimizados para o experimento. Isso é feito através do próprio comando de execução. Pode ser `'direct'` no qual a identificação dos parâmetros se dá pela ordem  
> ./run_exp 1 2 3  

Ou `'named'` com etiquetas
> ./run_exp --param1 1 --param2 2 --param3 3

As implementações por terminal e arquivos podem ser feitas por meio de um arquivo de configuração do M-LOOP, sem necessidade de um script Python.

#### `mloop.launchers`

Módulo dos launchers do M-LOOP. Estes não precisam ser chamados diretamente pelo usuário, ficando a cargo do controlador.

#### `mloop.learners`

Módulo dos algoritmos que fazem a seleção dos parâmetros. Os learners são criados e controlados pelo controlador. 

#### `mloop.testing`

Módulo de testes utilizado para verificar a instalação quando o usuário executa

#### `mloop.utilities`

Contém métodos e atributos comuns a todos os módulos.

#### `mloop.visualizations`

Classes para a criação de visualizações dos resultados dos experimentos e dos learners.